In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    # not going to use these
    N_DISTORTIONS = 0
    MAX_DISTORTION = 0.3
    MIN_DISTORTION = -0.1
    
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    # only flips
    IMAGE_AUGMENT = True

    MEAN_PIXEL = np.array([123.7, 116.8, 103.9],dtype=np.float32)
    MAX_GT_INSTANCES = 100
    DETECTION_MAX_INSTANCES = 100
    DETECTION_MIN_CONFIDENCE = 0.7
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    DETECTION_NMS_THRESHOLD = 0.3
    LEARNING_RATE = 0.05
    LEARNING_MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    WIDTH = 640
    HEIGHT = 640
    MASK_SHAPE = (64,64)
    GRID_WIDTH = 16
    GRID_HEIGHT = 16
    CLUE_SHAPE = (20,20)
    GRID_SHAPE = (GRID_WIDTH, GRID_HEIGHT)
    GRID_RESOLUTION = (1, 1)
    IS_PADDED = True
    MASK_THRESOLD = 0.7
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)
        self.MAX_BATCH_SIZE = self.BATCH_SIZE*32

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
root_dir = "/media/data/nishanth/aravind/"
config = ProposalConfig()
model_dir = "./models/"
train_dataset,val_dataset = None,None
train_pickle = root_dir+"val_cid.pickle"
val_pickle = root_dir+"val_cid.pickle"
class ClassInstancesDataset():
    def __init__(self):
        self.class_wise_instance_info = [[] for i in range(81)]
        self.instance_info = []
with open(train_pickle,"rb") as train_ann:
    train_cid = pickle.load(train_ann)
#     train_dataset = model_lib.CocoDataset(train_cid,config)
with open(val_pickle,"rb") as val_ann:
    val_cid = pickle.load(val_ann)
#     val_dataset = model_lib.CocoDataset(val_cid,config)
train_loader = model_lib.get_loader(train_cid,config)
val_loader = model_lib.get_loader(val_cid,config)

In [ ]:
net = model_lib.SimpleHGModel()
net.load_state_dict(torch.load(model_dir+"model_0_0.pt"))
device = torch.device("cuda:1")
net.to(device)
with torch.no_grad():
    for i,data in enumerate(train_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = data
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = batch_images.to(device),batch_impulses.to(device),batch_gt_responses.to(device),batch_class_ids.to(device)
        pred_class = net([batch_images,batch_impulses])
        maxs, indices = torch.max(pred_class, -1)
        print(batch_class_ids)
        print(indices)
        input()

In [4]:
import torch.optim as optim
import torch
net = model_lib.SimpleHGModel()
device = torch.device("cuda:1")
net.to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



net_size = sum([i.numel() for i in net.parameters()])
print(device)
print(net_size)

iters_per_checkpoint = 20
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i,data in enumerate(train_loader,0):
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = data
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = batch_images.to(device),batch_impulses.to(device),batch_gt_responses.to(device),batch_class_ids.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        pred_class = net([batch_images,batch_impulses])
        loss = model_lib.loss_criterion(0,0,batch_class_ids,pred_class)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() 
        if i % iters_per_checkpoint == 0:
            print("batch: ",i,"epoch: ",epoch, "loss: %0.5f" % (running_loss/iters_per_checkpoint))
            torch.save(net.state_dict(),model_dir+"model_%d_%d.pt"%(epoch,i))
            running_loss = 0.0
    # print("batch: %d time:%0.3f sec" %(i, end-start)); print(loss.item())
        


print('Finished Training')

cuda:1
383649
batch: 0 time:0.597 sec
4.552321434020996
batch: 1 time:0.057 sec
4.452633380889893
batch: 2 time:0.054 sec
4.230525970458984
batch: 3 time:0.047 sec
4.470552921295166
batch: 4 time:0.044 sec
4.250648498535156
batch: 5 time:0.049 sec
4.463964939117432
batch: 6 time:0.054 sec
4.3712568283081055
batch: 7 time:0.055 sec
4.465054035186768
batch: 8 time:0.043 sec
4.26613712310791
batch: 9 time:0.044 sec
4.430860996246338
batch: 10 time:0.053 sec
4.5314130783081055
batch: 11 time:0.048 sec
4.516486167907715
batch: 12 time:0.054 sec
4.360194683074951
batch: 13 time:0.067 sec
4.313287258148193
batch: 14 time:0.047 sec
4.557095527648926
batch: 15 time:0.058 sec
4.544212818145752
batch: 16 time:0.048 sec
4.435365200042725
batch: 17 time:0.045 sec
4.317532062530518
batch: 18 time:0.062 sec
4.532690048217773
batch: 19 time:0.051 sec
4.243348598480225
batch: 20 time:0.043 sec
4.464261054992676
batch: 21 time:0.063 sec
4.462587833404541
batch: 22 time:0.060 sec
4.437558174133301
batch:

4.2868876457214355
batch: 189 time:0.046 sec
4.473052978515625
batch: 190 time:0.056 sec
4.323592662811279
batch: 191 time:0.048 sec
4.486555576324463
batch: 192 time:0.051 sec
4.3339385986328125
batch: 193 time:0.050 sec
4.3787078857421875
batch: 194 time:0.077 sec
4.303182601928711
batch: 195 time:0.051 sec
4.312233924865723
batch: 196 time:0.052 sec
4.362563133239746
batch: 197 time:0.044 sec
4.363519191741943
batch: 198 time:0.049 sec
4.364880561828613
batch: 199 time:0.055 sec
4.4475016593933105
batch: 200 time:0.060 sec
4.438732624053955
batch: 201 time:0.047 sec
4.477157115936279
batch: 202 time:0.053 sec
4.279274940490723
batch: 203 time:0.046 sec
4.508884429931641
batch: 204 time:0.050 sec
4.4423089027404785
batch: 205 time:0.053 sec
4.327219486236572
batch: 206 time:0.047 sec
4.3387861251831055
batch: 207 time:0.064 sec
4.345936298370361
batch: 208 time:0.048 sec
4.438474655151367
batch: 209 time:0.049 sec
4.259535312652588
batch: 210 time:0.049 sec
4.35452938079834
batch: 21

4.407758712768555
batch: 376 time:0.051 sec
4.30558967590332
batch: 377 time:0.048 sec
4.386935234069824
batch: 378 time:0.057 sec
4.391686916351318
batch: 379 time:0.049 sec
4.341306686401367
batch: 380 time:0.049 sec
4.390059947967529
batch: 381 time:0.048 sec
4.163550853729248
batch: 382 time:0.046 sec
4.331081867218018
batch: 383 time:0.049 sec
4.3028082847595215
batch: 384 time:0.060 sec
4.427877902984619
batch: 385 time:0.064 sec
4.148604393005371
batch: 386 time:0.048 sec
4.223526477813721
batch: 387 time:0.059 sec
4.361802101135254
batch: 388 time:0.057 sec
4.182464599609375
batch: 389 time:0.060 sec
4.443589210510254
batch: 390 time:0.058 sec
4.348670959472656
batch: 391 time:0.057 sec
4.219048500061035
batch: 392 time:0.050 sec
4.406069755554199
batch: 393 time:0.051 sec
4.46923303604126
batch: 394 time:0.049 sec
4.3275017738342285
batch: 395 time:0.045 sec
4.354095458984375
batch: 396 time:0.054 sec
4.344641208648682
batch: 397 time:0.051 sec
4.270142555236816
batch: 398 tim

4.142965316772461
batch: 563 time:0.052 sec
4.456808090209961
batch: 564 time:0.052 sec
4.392285346984863
batch: 565 time:0.046 sec
4.303226947784424
batch: 566 time:0.051 sec
4.281321048736572
batch: 567 time:0.057 sec
4.384077548980713
batch: 568 time:0.048 sec
4.4987616539001465
batch: 569 time:0.050 sec
4.226905822753906
batch: 570 time:0.060 sec
4.33497953414917
batch: 571 time:0.052 sec
4.200024127960205
batch: 572 time:0.061 sec
4.204165935516357
batch: 573 time:0.061 sec
4.255369186401367
batch: 574 time:0.065 sec
4.369187831878662
batch: 575 time:0.051 sec
4.424798965454102
batch: 576 time:0.056 sec
4.136329650878906
batch: 577 time:0.052 sec
4.401398181915283
batch: 578 time:0.050 sec
4.242973804473877
batch: 579 time:0.054 sec
4.289545059204102
batch: 580 time:0.050 sec
4.3481879234313965
batch: 581 time:0.059 sec
4.293694496154785
batch: 582 time:0.050 sec
4.2811431884765625
batch: 583 time:0.048 sec
4.35772180557251
batch: 584 time:0.052 sec
4.2979607582092285
batch: 585 t

4.298462867736816
batch: 749 time:0.050 sec
4.429651737213135
batch: 750 time:0.054 sec
4.395691871643066
batch: 751 time:0.059 sec
4.269289970397949
batch: 752 time:0.050 sec
4.32662296295166
batch: 753 time:0.054 sec
4.207880973815918
batch: 754 time:0.062 sec
4.272722244262695
batch: 755 time:0.060 sec
4.356876850128174
batch: 756 time:0.056 sec
4.3328680992126465
batch: 757 time:0.046 sec
4.1942667961120605
batch: 758 time:0.049 sec
4.26450252532959
batch: 759 time:0.053 sec
4.114331245422363
batch: 760 time:0.055 sec
4.270007133483887
batch: 761 time:0.054 sec
4.256861209869385
batch: 762 time:0.045 sec
4.393072128295898
batch: 763 time:0.047 sec
4.436372756958008
batch: 764 time:0.054 sec
4.352311134338379
batch: 765 time:0.059 sec
4.384943962097168
batch: 766 time:0.055 sec
4.22989559173584
batch: 767 time:0.062 sec
4.295250415802002
batch: 768 time:0.047 sec
4.375385284423828
batch: 769 time:0.047 sec
4.246318340301514
batch: 770 time:0.049 sec
4.173991680145264
batch: 771 time

4.400507926940918
batch: 936 time:0.050 sec
4.371922969818115
batch: 937 time:0.050 sec
4.193722248077393
batch: 938 time:0.051 sec
4.470126152038574
batch: 939 time:0.046 sec
4.271200656890869
batch: 940 time:0.053 sec
4.26085090637207
batch: 941 time:0.058 sec
4.438516616821289
batch: 942 time:0.050 sec
4.263701438903809
batch: 943 time:0.056 sec
4.164536476135254
batch: 944 time:0.054 sec
4.237521648406982
batch: 945 time:0.049 sec
4.290870189666748
batch: 946 time:0.058 sec
4.260342121124268
batch: 947 time:0.054 sec
4.238745212554932
batch: 948 time:0.068 sec
4.3445210456848145
batch: 949 time:0.050 sec
4.396487236022949
batch: 950 time:0.046 sec
4.308823585510254
batch: 951 time:0.047 sec
4.007784843444824
batch: 952 time:0.053 sec
4.353194713592529
batch: 953 time:0.048 sec
4.1787638664245605
batch: 954 time:0.052 sec
4.28397274017334
batch: 955 time:0.046 sec
4.297050476074219
batch: 956 time:0.056 sec
4.519852638244629
batch: 957 time:0.044 sec
4.321290016174316
batch: 958 tim

In [5]:
from PIL import Image
import random
import time
np.set_printoptions(threshold=np.nan)
s = 0
for i,g in enumerate(train_loader):
#     if(i%2000 == 0):
    print("sample:" + str(i))
#         print(s)
    start = time.time()
#     print(g[0].shape)
    batch_images,batch_impulses,batch_gt_responses,batch_class_ids = g
    batch_images = np.moveaxis(batch_images.numpy(),1,-1)
    batch_impulses = batch_impulses.squeeze().numpy()
    batch_gt_responses = batch_gt_responses.squeeze().numpy()
    batch_class_ids = batch_class_ids.numpy()
#     print(batch_images.shape)
    s =(time.time()-start)
    print(s)
#     for j in range(batch_images.shape[0]):
#         img = Image.fromarray((batch_images[j]).astype("uint8"),"RGB")
#         img.show()
# #         print(batch_gt_responses[j])
#         mask = Image.fromarray(((batch_gt_responses[j])).astype("uint8"),"L")
#         mask.show()
#         print(config.CLASS_NAMES[batch_class_ids[j]])
#         impulse = Image.fromarray(((batch_impulses[j])).astype("uint8"),"L")
#         impulse.show()
#         input()
# idx = random.choice(range(len(train_dataset.image_ids)))
# print(len(train_dataset.image_ids))
# img = Image.fromarray(train_dataset.load_image(idx).astype("uint8"),"RGB")
# img.show()
# masks,class_ids = train_dataset.load_mask(idx)
# print(masks.shape)
#     input()

sample:0
0.0009703636169433594
sample:1
0.008966445922851562
sample:2
0.022534608840942383
sample:3
0.016726255416870117
sample:4
0.012122392654418945
sample:5
0.009214401245117188
sample:6
0.009888887405395508
sample:7
0.013018369674682617
sample:8
0.006815433502197266
sample:9
0.006075620651245117
sample:10
0.01522064208984375
sample:11
0.011278152465820312
sample:12
0.014945030212402344
sample:13
0.0076580047607421875
sample:14
0.011784553527832031
sample:15
0.01477670669555664
sample:16
0.016620874404907227
sample:17
0.009013652801513672
sample:18
0.013542890548706055
sample:19
0.00851893424987793
sample:20
0.008185625076293945
sample:21
0.008523702621459961
sample:22
0.008642196655273438
sample:23
0.008770227432250977
sample:24
0.009184598922729492
sample:25
0.012349367141723633
sample:26
0.014955520629882812
sample:27
0.0066127777099609375
sample:28
0.014948606491088867
sample:29
0.02278590202331543
sample:30
0.02152419090270996
sample:31
0.00869131088256836
sample:32
0.011420488

sample:267
0.03229117393493652
sample:268
0.009656429290771484
sample:269
0.0196225643157959
sample:270
0.00929403305053711
sample:271
0.013803482055664062
sample:272
0.01800370216369629
sample:273
0.00894784927368164
sample:274
0.013582229614257812
sample:275
0.018199920654296875
sample:276
0.013348102569580078
sample:277
0.018617630004882812
sample:278
0.010136127471923828
sample:279
0.017643213272094727
sample:280
0.02036118507385254
sample:281
0.014593839645385742
sample:282
0.008392095565795898
sample:283
0.014913320541381836
sample:284
0.007797718048095703
sample:285
0.009396076202392578
sample:286
0.007552385330200195
sample:287
0.015658855438232422
sample:288
0.007254838943481445
sample:289
0.01715373992919922
sample:290
0.013010025024414062
sample:291
0.007079362869262695
sample:292
0.006400346755981445
sample:293
0.005972146987915039
sample:294
0.01908278465270996
sample:295
0.015268802642822266
sample:296
0.010183572769165039
sample:297
0.030389785766601562
sample:298
0.0250

sample:526
0.011120080947875977
sample:527
0.008745431900024414
sample:528
0.009113788604736328
sample:529
0.011842489242553711
sample:530
0.013464689254760742
sample:531
0.013829946517944336
sample:532
0.008246660232543945
sample:533
0.00852060317993164
sample:534
0.015521049499511719
sample:535
0.01034855842590332
sample:536
0.013730049133300781
sample:537
0.013696908950805664
sample:538
0.006415843963623047
sample:539
0.019490718841552734
sample:540
0.007348299026489258
sample:541
0.0064411163330078125
sample:542
0.007808208465576172
sample:543
0.007676601409912109
sample:544
0.00905919075012207
sample:545
0.039134979248046875
sample:546
0.026422739028930664
sample:547
0.007122993469238281
sample:548
0.017479419708251953
sample:549
0.016852378845214844
sample:550
0.020489215850830078
sample:551
0.007936954498291016
sample:552
0.009604930877685547
sample:553
0.008957862854003906
sample:554
0.008522272109985352
sample:555
0.00930023193359375
sample:556
0.008975028991699219
sample:557


sample:784
0.01562809944152832
sample:785
0.007317781448364258
sample:786
0.006905078887939453
sample:787
0.010017156600952148
sample:788
0.008537530899047852
sample:789
0.008919239044189453
sample:790
0.016458511352539062
sample:791
0.014470100402832031
sample:792
0.020157814025878906
sample:793
0.020447731018066406
sample:794
0.031880855560302734
sample:795
0.008038043975830078
sample:796
0.017459630966186523
sample:797
0.008259296417236328
sample:798
0.01935410499572754
sample:799
0.008034467697143555
sample:800
0.007317781448364258
sample:801
0.00886988639831543
sample:802
0.016632795333862305
sample:803
0.010594606399536133
sample:804
0.012607336044311523
sample:805
0.02221059799194336
sample:806
0.027190685272216797
sample:807
0.008349895477294922
sample:808
0.016133785247802734
sample:809
0.015531539916992188
sample:810
0.011249542236328125
sample:811
0.010442495346069336
sample:812
0.011277198791503906
sample:813
0.0071315765380859375
sample:814
0.009877920150756836
sample:815


sample:1042
0.011907815933227539
sample:1043
0.008997917175292969
sample:1044
0.018818378448486328
sample:1045
0.006104230880737305
sample:1046
0.006884336471557617
sample:1047
0.007158517837524414
sample:1048
0.008501291275024414
sample:1049
0.015253067016601562
sample:1050
0.010110855102539062
sample:1051
0.011570453643798828
sample:1052
0.017275094985961914
sample:1053
0.02675771713256836
sample:1054
0.017249584197998047
sample:1055
0.021793365478515625
sample:1056
0.009278297424316406
sample:1057
0.009177207946777344
sample:1058
0.011670351028442383
sample:1059
0.016283750534057617
sample:1060
0.010010957717895508
sample:1061
0.021019458770751953
sample:1062
0.014021635055541992
sample:1063
0.0107879638671875
sample:1064
0.019318819046020508
sample:1065
0.008172750473022461
sample:1066
0.017723560333251953
sample:1067
0.009839534759521484
sample:1068
0.017229557037353516
sample:1069
0.01766371726989746
sample:1070
0.00840306282043457
sample:1071
0.009657859802246094
sample:1072
0.0

...

sample:4082
0.009739875793457031
sample:4083
0.008453369140625
sample:4084
0.011440277099609375
sample:4085
0.00950932502746582
sample:4086
0.008982658386230469
sample:4087
0.013611316680908203
sample:4088
0.009026050567626953
sample:4089
0.010408401489257812
sample:4090
0.009565591812133789
sample:4091
0.00916147232055664
sample:4092
0.011211156845092773
sample:4093
0.012740612030029297
sample:4094
0.011614799499511719
sample:4095
0.008768796920776367
sample:4096
0.011773824691772461
sample:4097
0.00990605354309082
sample:4098
0.024707794189453125
sample:4099
0.01743626594543457
sample:4100
0.0069751739501953125
sample:4101
0.0071871280670166016
sample:4102
0.010157108306884766
sample:4103
0.01003408432006836
sample:4104
0.010796546936035156
sample:4105
0.010032176971435547
sample:4106
0.009045124053955078
sample:4107
0.00789332389831543
sample:4108
0.009773969650268555
sample:4109
0.00983572006225586
sample:4110
0.02537083625793457
sample:4111
0.009523391723632812
sample:4112
0.00973

sample:4332
0.008154630661010742
sample:4333
0.007478475570678711
sample:4334
0.012753725051879883
sample:4335
0.011214733123779297
sample:4336
0.008473873138427734
sample:4337
0.007460355758666992
sample:4338
0.008221864700317383
sample:4339
0.009250164031982422
sample:4340
0.008085012435913086
sample:4341
0.008618831634521484
sample:4342
0.00842595100402832
sample:4343
0.009682893753051758
sample:4344
0.015220165252685547
sample:4345
0.008194923400878906
sample:4346
0.00785517692565918
sample:4347
0.00783991813659668
sample:4348
0.009324312210083008
sample:4349
0.009074687957763672
sample:4350
0.009531736373901367
sample:4351
0.01048898696899414
sample:4352
0.009464263916015625
sample:4353
0.009523630142211914
sample:4354
0.009026527404785156
sample:4355
0.008573293685913086
sample:4356
0.008621931076049805
sample:4357
0.009118318557739258
sample:4358
0.011216878890991211
sample:4359
0.009845495223999023
sample:4360
0.009449958801269531
sample:4361
0.008149147033691406
sample:4362
0.

sample:4582
0.010286808013916016
sample:4583
0.006602048873901367
sample:4584
0.0064754486083984375
sample:4585
0.007504463195800781
sample:4586
0.025079727172851562
sample:4587
0.007470130920410156
sample:4588
0.00793147087097168
sample:4589
0.00969076156616211
sample:4590
0.01190042495727539
sample:4591
0.01130986213684082
sample:4592
0.00883173942565918
sample:4593
0.01018214225769043
sample:4594
0.009765863418579102
sample:4595
0.008783102035522461
sample:4596
0.012189626693725586
sample:4597
0.009861946105957031
sample:4598
0.009880781173706055
sample:4599
0.008652687072753906
sample:4600
0.009724855422973633
sample:4601
0.025072813034057617
sample:4602
0.010591268539428711
sample:4603
0.009402990341186523
sample:4604
0.00974273681640625
sample:4605
0.008882761001586914
sample:4606
0.008004426956176758
sample:4607
0.007127046585083008
sample:4608
0.006674051284790039
sample:4609
0.0073664188385009766
sample:4610
0.007994651794433594
sample:4611
0.006595611572265625
sample:4612
0.0

sample:4833
0.011292219161987305
sample:4834
0.013418436050415039
sample:4835
0.009879827499389648
sample:4836
0.007998943328857422
sample:4837
0.016163349151611328
sample:4838
0.021070241928100586
sample:4839
0.01488804817199707
sample:4840
0.009047269821166992
sample:4841
0.008489131927490234
sample:4842
0.009921789169311523
sample:4843
0.009239435195922852
sample:4844
0.010695934295654297
sample:4845
0.009855031967163086
sample:4846
0.009793996810913086
sample:4847
0.01161336898803711
sample:4848
0.011296272277832031
sample:4849
0.007226228713989258
sample:4850
0.009982824325561523
sample:4851
0.011797189712524414
sample:4852
0.008844375610351562
sample:4853
0.009963750839233398
sample:4854
0.009392976760864258
sample:4855
0.009676694869995117
sample:4856
0.008903264999389648
sample:4857
0.007276773452758789
sample:4858
0.014420509338378906
sample:4859
0.00885009765625
sample:4860
0.008869409561157227
sample:4861
0.011270999908447266
sample:4862
0.012882471084594727
sample:4863
0.01

sample:5086
0.008370399475097656
sample:5087
0.007454395294189453
sample:5088
0.009367704391479492
sample:5089
0.009930133819580078
sample:5090
0.012103080749511719
sample:5091
0.007279396057128906
sample:5092
0.007467985153198242
sample:5093
0.017219066619873047
sample:5094
0.01116490364074707
sample:5095
0.010754823684692383
sample:5096
0.010169029235839844
sample:5097
0.009311914443969727
sample:5098
0.011057615280151367
sample:5099
0.009562969207763672
sample:5100
0.009937047958374023
sample:5101
0.02667999267578125
sample:5102
0.0097808837890625
sample:5103
0.009555578231811523
sample:5104
0.012798070907592773
sample:5105
0.011703729629516602
sample:5106
0.010882139205932617
sample:5107
0.010710954666137695
sample:5108
0.008978128433227539
sample:5109
0.010119199752807617
sample:5110
0.011647462844848633
sample:5111
0.010613679885864258
sample:5112
0.012872934341430664
sample:5113
0.012281417846679688
sample:5114
0.030022144317626953
sample:5115
0.01033329963684082
sample:5116
0.0